In [6]:
from utils.key import *
from utils.db import *
from utils.match import *

In [8]:
from tqdm import tqdm,trange

BIN_FUNC_DB = './step2_postSig/arm/bin_func.db'
conn = sqlite3.connect(BIN_FUNC_DB)
cursor = conn.cursor()

cursor.execute('SELECT COUNT(*) FROM func_table')
result_len = cursor.fetchone()[0]

BASE_DB = './match_base/func.db'
conn1 = sqlite3.connect(BASE_DB)
cursor1 = conn1.cursor()

cursor.execute('SELECT * FROM func_table')
all_results = {}
max_results = {}
for i in trange(result_len):
    result_ = cursor.fetchone()
    if result_ is None:
        tqdm.write(f'wrong')
        break 
    # del the id 
    result_ = result_[1:]
    name_ = result_[func_key_idx('name')]
    program_ = result_[func_key_idx('program')]
    (max_, matches_) = compare_func_db(cursor1, result_)
    if len(matches_) > 0:
        if all_results.get(program_) is None:
            all_results[program_] = {}
            max_results[program_] = {}
        match_result_ = []
        for match_ in matches_:
            # only add func name and program name and numAddresses and ratio to result_
            match_result_.append([match_[0][0],match_[0][1],match_[0][3],match_[1]])
        all_results[program_][name_] = match_result_
        # max results 
        max_results[program_][name_] = {'name':max_[0][0],
                                    'program': max_[0][1],
                                    'numAddr': max_[0][3],
                                    'ratio': max_[1]}
conn.close()
conn1.close()

100%|█████████████████████████████████████████████████████████████████████████| 134823/134823 [16:46<00:00, 133.92it/s]


In [9]:
import json
with open('./func_match.json', 'w') as file:
    json.dump(all_results, file, indent=4)
with open('./func_max_match.json', 'w') as file:
    json.dump(max_results, file, indent=4)

In [10]:
with open('match_program.csv', 'w') as file:
    for (k,v) in max_results.items():
        file.write(f'{k},{len(v)}\n')

# END

In [ ]:
# Not use now, match the functions 
from ghidra.program.flatapi import FlatProgramAPI
from tqdm import tqdm
DATABASE = './match_base/func.db'
conn = sqlite3.connect(DATABASE)
FUNC_TABLE_NAME = 'func_table'
cursor = conn.cursor()
all_results_ = {}
# collect file name for tqdm 
#file_names = []
#for file_ in project.getRootFolder().getFiles():
 #   file_names.append(file_.getName())
# use tqdm to generate the progress bar 
for file_ in tqdm(project.getRootFolder().getFiles()):
    # print(f"Match program {name_}")
    name_ = file_.getName()
    program = project.openProgram('/', name_, True)
    match_result_ = {}
    for func_ in program.getListing().getFunctions(True):
            if filter_func(func_):
                (max_, matches_) = compare_func(cursor, func_)
                if len(matches_) > 0:
                    match_result_[func_.getName()] = []
                    for match_ in matches_:
                        # only add func name and ratio to result_
                        match_result_[func_.getName()].append([match_[0][1],match_[1]])
                else:
                    continue 
                    print(f'No match {func_.getName()}')
    # remember closing the program to avoid memory usage 
    project.close(program)
    if len(match_result_) > 0:
        tqdm.write(f"\033[31m{name_} has match {len(match_result_)} times\033[0m")
        all_results_[name_] = match_result_
conn.close()

In [51]:
a = 'a'
str(type(a))

"<class 'str'>"

In [8]:
import sqlite3
BIN_FUNC_DB = './step2_postSig/arm/bin_func.db'
conn = sqlite3.connect(BIN_FUNC_DB)
cursor = conn.cursor()

cursor.execute('SELECT COUNT(*)ROM func_table')
result = cursor.fetchone()

In [9]:
result[0]

10742545